In [ ]:
from appgeopy import *
from my_packages import *

#### nscore-transformed displacement

In [ ]:
def backtransform_func(
    original_df, transformed_df, original_df_indexcol, cols2transform
):
    import geostatspy.geostats as geostats
    import geostatspy.GSLIB as GSLIB
    from tqdm import tqdm

    output_df = transformed_df.loc[
        :, ~transformed_df.columns.isin(cols2transform)
    ].copy()

    for col in tqdm(cols2transform):
        temp = (
            original_df[[original_df_indexcol, col]].dropna(subset=[col]).copy()
        )
        init_transformed, sorted_init, sorted_weighting = geostats.nscore(
            temp, col
        )
        # Performing back-transformation using geostatspy
        output_df[col] = geostats.backtr(
            df=transformed_df,
            vcol=col,
            vr=sorted_init,
            vrg=sorted_weighting,
            zmin=-1e21,
            zmax=1e21,
            ltail=0,
            ltpar=0,
            utail=60,
            utpar=60,
        )

    return output_df

In [ ]:
# transformed data obtained from Kriging interpolation
transformed_data_fpath = (
    r"NSCORE_CORRECTED_Monthly_DISPLACEMENT_CRFP_saveqgis_Oct2025_grid.xz"
)
transformed_df = pd.read_pickle(transformed_data_fpath)
transformed_df.shape

In [ ]:
# read the original data, from which the transformed data was achieved
# original_data_fpath = r"D:\1000_SCRIPTS\003_Project002\20250917_GTWR002\2_KrigingInterpolation\Monthly_DISPLACEMENT_dU_CRFP_2025.xz"
original_data_fpath = r"D:\1000_SCRIPTS\003_Project002\20250917_GTWR002\2_KrigingInterpolation\CORRECTED_Monthly_DISPLACEMENT_CRFP_saveqgis_Oct2025.xz"
original_df = pd.read_pickle(original_data_fpath)
original_df = original_df.reset_index(drop=False)
trans_cols = [col for col in original_df.columns if col.startswith("N")]

# perform backtransform
backtransformed_df = backtransform_func(
    original_df,
    transformed_df,
    original_df_indexcol="PointKey",
    cols2transform=trans_cols,
)

In [ ]:
# save output to pickle
transformed_data_basename = os.path.basename(transformed_data_fpath)

backtransformed_savename = transformed_data_basename.replace(
    "NSCORE", "BACKTRANSFORMED"
)

# grid or mlcw???
out_fld = "grid"

backtransformed_savepath = os.path.join(
    "PostKriging_Backtransform_Data",
    backtransformed_savename.replace(".xz", f"_{out_fld}.xz"),
)

if out_fld == "mlcw":
    # regarding the MLCW points, before saving it to pickle
    # we need to add back the `STATION` and `WellCode` column

    # we will borrow the old file to get the information
    fpath = r"D:\1000_SCRIPTS\003_Project002\20250222_GTWR001\2_KrigingInterpolation\5_PostKriging\PostKriging_Backtransform_Data\mlcw_points\BACKTRANSFORMED_Monthly_DISPLACEMENT_dU_CRFP_Full.xz"
    temp = pd.read_pickle(fpath)

    # I need to create a PointKey column to match with the `backtransformed_df`
    pointkey_arr = [
        f"X{int(x*1000)}Y{int(y*1000)}"
        for x, y in zip(temp["X_TWD97"], temp["Y_TWD97"])
    ]
    temp.insert(loc=0, column="PointKey", value=pointkey_arr)
    temp = temp.reset_index(drop=False)
    temp = temp.set_index("PointKey")

    # add the `STATION` and `WellCode` column into the `backtransformed_df`,
    # then rearrange the columns
    for col in ["STATION", "WellCode"]:
        backtransformed_df[col] = temp.index.map(temp[col])

    backtransformed_df = backtransformed_df.reset_index(drop=False)

    new_cols = ["STATION", "WellCode", "X_TWD97", "Y_TWD97"] + [
        col for col in backtransformed_df.columns if col.startswith("N")
    ]
    backtransformed_df = backtransformed_df[new_cols]
    backtransformed_df = backtransformed_df.set_index("STATION")

backtransformed_df.to_pickle(backtransformed_savepath)

In [ ]:
show(backtransformed_df)